In [1]:
import pandas as pd

In [2]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

### Optimizing dataframe memory footprint

Let's assume that we have a limited PC with very limited memory. In this exercise, we will not use more than 5 MB of RAM memory while looping through the dataset. First, let's do some EDA with the first few rows and find out the total memory usage of the dataframe.

In [3]:
csv_loc = 'https://raw.githubusercontent.com/koen-d-r/optimizing_dataframe_memory/master/loans_2007.csv'
df_test = pd.read_csv(csv_loc, nrows = 5)
print(df_test.info())
df_test.head()
original_col_names = df_test.columns.tolist()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 52 columns):
id                            5 non-null int64
member_id                     5 non-null float64
loan_amnt                     5 non-null float64
funded_amnt                   5 non-null float64
funded_amnt_inv               5 non-null float64
term                          5 non-null object
int_rate                      5 non-null object
installment                   5 non-null float64
grade                         5 non-null object
sub_grade                     5 non-null object
emp_title                     3 non-null object
emp_length                    5 non-null object
home_ownership                5 non-null object
annual_inc                    5 non-null float64
verification_status           5 non-null object
issue_d                       5 non-null object
loan_status                   5 non-null object
pymnt_plan                    5 non-null object
purpose                       

In [4]:
# find the number of dtypes in df_test
df_test.dtypes.value_counts()

float64    30
object     21
int64       1
dtype: int64

In [5]:
# rewrite column names to column name + dtype for understanding the dtypes and underlying data better
df_test.columns = [df_test.columns[num] + '_' + str(df_test.dtypes[num]) for num in range(df_test.shape[1])]

In [6]:
df_test.head()

,id_int64,member_id_float64,loan_amnt_float64,funded_amnt_float64,funded_amnt_inv_float64,term_object,int_rate_object,installment_float64,grade_object,sub_grade_object,emp_title_object,emp_length_object,home_ownership_object,annual_inc_float64,verification_status_object,issue_d_object,loan_status_object,pymnt_plan_object,purpose_object,title_object,zip_code_object,addr_state_object,dti_float64,delinq_2yrs_float64,earliest_cr_line_object,inq_last_6mths_float64,open_acc_float64,pub_rec_float64,revol_bal_float64,revol_util_object,total_acc_float64,initial_list_status_object,out_prncp_float64,out_prncp_inv_float64,total_pymnt_float64,total_pymnt_inv_float64,total_rec_prncp_float64,total_rec_int_float64,total_rec_late_fee_float64,recoveries_float64,collection_recovery_fee_float64,last_pymnt_d_object,last_pymnt_amnt_float64,last_credit_pull_d_object,collections_12_mths_ex_med_float64,policy_code_float64,application_type_object,acc_now_delinq_float64,chargeoff_within_12_mths_float64,delinq_amnt_float64,pub_rec_bankruptcies_float64,tax_liens_float64
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-2015,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-2014,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-2015,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,461.73,461.73,3581.120000,3581.12,2538.27,1042.85,0.00,0.00,0.00,Jun-2016,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


### Memory constraint
Let's import the data into chunks. Let's find out what the maximum number of rows in a chunk is!

In [7]:
memory_constraint = 5 # 5MB that is!
memory_use = df_test.memory_usage(deep = True).sum() / 2 ** 20
max_chunk_size = (len(df_test) * memory_constraint) / memory_use

print(f'The df_test has {df_test.shape[0]} rows and {df_test.shape[1]} columns. With a 5 MB memory constraint, we can process {int(max_chunk_size)} rows in each chunk. Let\'s round that off to {(int(max_chunk_size) // 1000) * 1000} rows per chunk, just to be on the safe side.')

The df_test has 5 rows and 52 columns. With a 5 MB memory constraint, we can process 3257 rows in each chunk. Let's round that off to 3000 rows per chunk, just to be on the safe side.


In [8]:
df_chunks = pd.read_csv(csv_loc, chunksize = 3000)
total_memory_usage = 0
total_lines = 0
max_chunk_size = 0

for chunk in df_chunks:
    size = chunk.memory_usage(deep = True).sum() / 2 ** 20
    total_memory_usage += size
    total_lines += len(chunk)
    if max_chunk_size < size:
        max_chunk_size = size

print(total_memory_usage)
print(total_lines)
print(max_chunk_size)

65.72928524017334
42538
4.862635612487793


In [9]:
print(f'The total dataframe measures {round(total_memory_usage, 1)} MB and contains {df_test.shape[1]} columns and {total_lines} rows.')

The total dataframe measures 65.7 MB and contains 52 columns and 42538 rows.


### Finding columns to change to categorical columns

While looping through the dataframe chunks, we will count all values with the value_counts() method and save the results in a dictionary. If a column contains less than 50 unique values, then we will add the column name to a list for later conversion.

In [10]:
df_chunks = pd.read_csv(csv_loc, chunksize = 3000)
val_counts_dict = {}

for chunk in df_chunks:
    chunk = chunk.select_dtypes('object')
    for col in chunk.columns:
        val_counts = chunk[col].value_counts()
        if col in val_counts_dict:
            val_counts_dict[col].append(val_counts)
        else:
            val_counts_dict[col] = [val_counts]

In [11]:
categorical_cols_list = []

for key in val_counts_dict.keys():
    shape = pd.concat(val_counts_dict[key]).groupby(level = 0).sum().shape[0]
    if shape < 50:
        categorical_cols_list.append(key)

In [12]:
# check results
categorical_cols_list[:5]

['term', 'grade', 'sub_grade', 'emp_length', 'home_ownership']

In [13]:
# use dict comprehension to add all categorical columns in a dict for import
col_dtype_dict = {x: 'category' for x in categorical_cols_list}

### Select column with dates to parse to datetime columns


In [14]:
dates = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d']

### Downcast numerical columns to smallest numerical dtype

Let's see if all columns can be downcasted to the same numerical dtype (e.g. np.int16) while iterating over the various chunks.

In [15]:
df_chunks = pd.read_csv(csv_loc, chunksize = 3000)

dtype_dict = {}

for chunk in df_chunks:
    chunk_float = chunk.select_dtypes(include = 'float')
    chunk_int = chunk.select_dtypes(include = 'integer')
    for col in chunk_float.columns:
        float_dtype = pd.to_numeric(chunk_float[col], downcast = 'float').dtype
        if col in dtype_dict:
            dtype_dict[col].add(float_dtype)
        else:
            dtype_dict[col] = {float_dtype} # using set to have better readability of results
    for col in chunk_int.columns:
        int_dtype = pd.to_numeric(chunk_int[col], downcast = 'float').dtype
        if col in dtype_dict:
            dtype_dict[col].add(int_dtype)
        else:
            dtype_dict[col] = {int_dtype} # using set to have better readability of results

In [16]:
dtype_dict

{'member_id': {dtype('float32')},
 'loan_amnt': {dtype('float32')},
 'funded_amnt': {dtype('float32')},
 'funded_amnt_inv': {dtype('float32')},
 'installment': {dtype('float32')},
 'annual_inc': {dtype('float32')},
 'dti': {dtype('float32')},
 'delinq_2yrs': {dtype('float32')},
 'inq_last_6mths': {dtype('float32')},
 'open_acc': {dtype('float32')},
 'pub_rec': {dtype('float32')},
 'revol_bal': {dtype('float32')},
 'total_acc': {dtype('float32')},
 'out_prncp': {dtype('float32')},
 'out_prncp_inv': {dtype('float32')},
 'total_pymnt': {dtype('float32')},
 'total_pymnt_inv': {dtype('float32')},
 'total_rec_prncp': {dtype('float32')},
 'total_rec_int': {dtype('float32')},
 'total_rec_late_fee': {dtype('float32')},
 'recoveries': {dtype('float32')},
 'collection_recovery_fee': {dtype('float32')},
 'last_pymnt_amnt': {dtype('float32')},
 'collections_12_mths_ex_med': {dtype('float32')},
 'policy_code': {dtype('float32')},
 'acc_now_delinq': {dtype('float32')},
 'chargeoff_within_12_mths': {d

All downcasted dtypes are consistent across the chunks in the iteration!

In [17]:
# remove unnecessary columns
usecols = [e for e in original_col_names if e not in ['id', 'empt_title']]

In [18]:
df_chunks = pd.read_csv(csv_loc, chunksize = 3000, parse_dates = dates, dtype = col_dtype_dict, usecols = usecols)
new_memory_usage = 0

for chunk in df_chunks:
    chunk['int_rate'] = pd.to_numeric(chunk['int_rate'].str.rstrip('%'), downcast = 'float')
    chunk['term'] = pd.to_numeric(chunk['term'].str.lstrip(' ').str.rstrip(' months'), downcast = 'integer')
    chunk['revol_util'] = pd.to_numeric(chunk['revol_util'].str.rstrip('%'), downcast = 'float')
    for float_col in chunk.select_dtypes('float').columns:
        chunk[float_col] = pd.to_numeric(chunk[float_col], downcast = 'float')
    for int_col in chunk.select_dtypes('integer').columns:
        chunk[int_col] = pd.to_numeric(chunk[int_col], downcast = 'integer')
    new_memory_usage += chunk.memory_usage(deep = True).sum() / 2 ** 20
    
print(new_memory_usage)

17.933223724365234


In [19]:
print(f'After the conversion, the total dataframe size is {round(new_memory_usage, 1)} MB. In the old situation, the dataframe was {round(total_memory_usage, 1)} MB. This is a decrease of {round(abs(((new_memory_usage - total_memory_usage) / total_memory_usage)) * 100, 1)}%. With dropping of other unnecessary columns, further decrease in memory use can be realized.')

After the conversion, the total dataframe size is 17.9 MB. In the old situation, the dataframe was 65.7 MB. This is a decrease of 72.7%. With dropping of other unnecessary columns, further decrease in memory use can be realized.
